In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#Example ADS-B data gathered from flightradar24
flight = pd.read_csv('data/24299085.csv')

print(list(flight))

In [ ]:
#We can use simple accessor methods to find out useful data
flight['Altitude'].min(),flight['Altitude'].max()

In [ ]:
#We can do more complex operations like finding the difference between each successive pair of value
flight['diff'] = flight['Altitude'].diff()

In [ ]:
flight['diff'].min(),flight['diff'].max()

In [ ]:
#We can also create functions to apply to our data
def energy(speed,altitude,mass,gravity=9.81):
    ke=0.5*mass*speed**2
    pe=mass*gravity*altitude
    return ke+pe

In [ ]:
#Using 5000kg as reasonably wrong answer for Jetstream 31
print(energy(flight['Speed'],flight['Altitude'],5000))

In [ ]:
#We can also apply it in a vectorised fashion on our pandas dataframe which is more efficent if your data is large
mass=5000
gravity=9.81
flight['ke'] = flight.apply(lambda r: 0.5*mass*r['Speed']**2,axis=1)
flight['pe'] = flight.apply(lambda r: gravity*mass*r['Altitude'],axis=1)

In [ ]:
flight.head()

In [ ]:
flight['Datetime'] = pd.to_datetime(flight['UTC'],utc=True)

In [ ]:
fig, axes = plt.subplots(3,1,sharex='all')
axes[0].plot(flight['Datetime'],flight['Altitude'])
axes[0].set_ylabel('Altitude')
axes[1].plot(flight['Datetime'],flight['Speed'])
axes[1].set_ylabel('Speed')
axes[2].plot(flight['Datetime'],flight['Direction'])
axes[2].set_ylabel('Direction')
axes[2].set_xlabel('UTC')
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(4,1,sharex='all')
axes[0].plot(flight['Datetime'],flight['Altitude'])
axes[0].set_ylabel('Alt')
axes[1].plot(flight['Datetime'],flight['Speed'])
axes[1].set_ylabel('Speed')
axes[2].plot(flight['Datetime'],flight['ke'])
axes[2].set_ylabel('KE')
axes[3].plot(flight['Datetime'],flight['pe'])
axes[3].set_ylabel('PE')
axes[3].set_xlabel('UTC')
fig.tight_layout()
plt.show()

In [ ]:
coords = [[float(b) for b in a.split(',')] for a in flight['Position']]
#This is a nest list comphrehension to convert the paired coordinates from their 
# comma seperated string format to a pair of floats

In [ ]:
flight['latitude'] = [a for a,b in coords]

In [ ]:
flight['longitude'] = [b for a,b in coords]

In [ ]:
%pip install -q folium

In [ ]:
import folium

m = folium.Map()
folium.PolyLine(zip(flight['latitude'],flight['longitude']), color="red", weight=2.5, opacity=1).add_to(m)
folium.LayerControl().add_to(m)
m.fit_bounds(m.get_bounds())
m

## Data Loading example


File Format

.csv

Line 

0  = metadata (File date, aircraft reg, etc)

1  = more metadata

2  =  Field names

3  =  Field Abbreviations

4  =  Units

5  =  Data Start

Where a variable is recorded at higher than the frame rate it is presented in multiple sequential columns. 
 
The field name is listed for the first column and subsequent columns have a blank field name.

However on lines 2 and 3 the abbreviation and units for the field are listed for each column.

//////  = nan

XXXXX   = nan

******  = nan

------  = nan

In [ ]:
frame_freq = 0.25

flight_data = pd.read_csv('data/FDM/FDM_data_01.csv',
                          skiprows=5,
                          na_values=['//////',
                                     'XXXXX',
                                     '******',
                                     '------'],
                          names=['Frame-SF',
                                 'Time',
                                 'Status',
                                 'PRESS ALT (ft)',
                                 'COMP AIRSPD DADC 1 (kts)',
                                 'EXCESS CABIN ALT',
                                 'N1 ENG 1 (% RPM)',
                                 'N1 ENG 2 (% RPM)',
                                 "MASTER WARN'G",
                                 'SPEED BRAKE +0s',
                                 'SPEED BRAKE +0.5s',
                                 'AT #1 ENGAGE',
                                 'AT #2 ENGAGE'],
                        )

In [ ]:
def frame_exact_timing(frame_sf_data,time_data,frame_rate):
    frame_name = frame_sf_data.name
    time_name = time_data.name
    dataframe = pd.DataFrame({frame_name:frame_sf_data,time_name:time_data})
    dataframe['changed'] = dataframe[time_name] == dataframe[time_name].shift(1).fillna(dataframe[time_name])
    min_changes = dataframe[frame_name][dataframe['changed']==False].index
    dataframe['seconds'] = 0 
    first_change=min_changes[0]
    for a in range(1,60):
        try:
            dataframe['seconds'].loc[first_change-a] = 60-a
        except IndexError as ie:
            print(ie)
                
        for b in min_changes:
            try:
                dataframe['seconds'].loc[b+a] = a
            except IndexError as ie:
                print(ie)
    
    return dataframe['seconds']

In [ ]:
flight_data['Frame'] = flight_data.apply(lambda r : r['Frame-SF'][:5],1)
flight_data['Subframe'] = flight_data.apply(lambda r : r['Frame-SF'][6:],1)
flight_data['s past min'] = frame_exact_timing(flight_data['Frame-SF'],flight_data['Time'],frame_freq)
flight_data['Time'] = flight_data.apply(lambda r : r['Time'] + ':' +str(r['s past min']),1)
flight_data['Time (str)'] = flight_data['Time']
flight_data['Time'] = pd.to_datetime(flight_data['Time'], format='%H:%M:%S').dt.time
del flight_data['s past min']

## Try plotting the Altitude and Airspeed using Matplotlib

## Now lets look at using SQL

https://docs.python.org/3/library/sqlite3.html

https://docs.sqlalchemy.org/en/14/dialects/sqlite.html

https://www.designcise.com/web/tutorial/what-is-the-order-of-execution-of-an-sql-query

In [ ]:
import sqlite3
import sqlalchemy

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
flight_data.to_sql('flight', con=engine)

SQL syntax is not case sensitive but often it is written with commands all caps.  This is a stylistic preference.  Not a requirement

In [ ]:
engine.execute("SELECT * FROM flight").fetchall()

We can find the unique entries in a particular field

In [ ]:
engine.execute("""SELECT distinct "SPEED BRAKE +0s" from flight;""").fetchall()

In [ ]:
engine.execute("""SELECT distinct "AT #1 ENGAGE" from flight;""").fetchall()